In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans,MiniBatchKMeans
from sklearn.model_selection import train_test_split,KFold,RandomizedSearchCV
 
# Models
from sklearn.ensemble import StackingRegressor
from lightgbm import LGBMRegressor

import matplotlib.pyplot as plt
import seaborn as sns
import folium
sns.set(style='darkgrid')
pd.set_option('display.max_columns', None)

no change     /Users/vrushalmantri/opt/anaconda3/condabin/conda
no change     /Users/vrushalmantri/opt/anaconda3/bin/conda
no change     /Users/vrushalmantri/opt/anaconda3/bin/conda-env
no change     /Users/vrushalmantri/opt/anaconda3/bin/activate
no change     /Users/vrushalmantri/opt/anaconda3/bin/deactivate
no change     /Users/vrushalmantri/opt/anaconda3/etc/profile.d/conda.sh
no change     /Users/vrushalmantri/opt/anaconda3/etc/fish/conf.d/conda.fish
no change     /Users/vrushalmantri/opt/anaconda3/shell/condabin/Conda.psm1
no change     /Users/vrushalmantri/opt/anaconda3/shell/condabin/conda-hook.ps1
no change     /Users/vrushalmantri/opt/anaconda3/lib/python3.9/site-packages/xontrib/conda.xsh
no change     /Users/vrushalmantri/opt/anaconda3/etc/profile.d/conda.csh
no change     /Users/vrushalmantri/.zshrc
No action taken.

CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Curre

OSError: dlopen(/Users/vrushalmantri/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: <3AF1EF0C-311C-31EC-BCE3-679F37ABEE16> /Users/vrushalmantri/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file, not in dyld cache)

* **id** - a unique identifier for each trip
* **vendor_id** - a code indicating the provider associated with the trip record
* **pickup_datetime** - date and time when the meter was engaged
* **dropoff_datetime** - date and time when the meter was disengaged
* **passenger_count** - the number of passengers in the vehicle (driver entered value)
* **pickup_longitude** - the longitude where the meter was engaged
* **pickup_latitude** - the latitude where the meter was engaged
* **dropoff_longitude** - the longitude where the meter was disengaged
* **dropoff_latitude** - the latitude where the meter was disengaged
* **store_and_fwd_flag** - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip
* **trip_duration** - duration of the trip in seconds

In [ ]:
train = pd.read_csv('/Users/vrushalmantri/Downloads/train (1).zip')
test = pd.read_csv('/Users/vrushalmantri/Downloads/test (1).zip')
print(f'Total train rows: {train.shape[0]}')
print(f'Total test rows: {test.shape[0]}')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.describe(include = 'all')

* id column has all unique values for both train and test sets
* vendor_id has only 2 unique values (1 or 2) with a median of 2
* passenger_count has 9 unique values (0,1,2,3,4,5,6,7,8 or 9) with a median of 1
* store_and_fwd_flag has 2 unique values (Y or N)

In [ ]:
# check for missing values -> No missing values found
print('Checking for missing data in train:')
print('Missing values found:',train.isna().sum().sum())
print('-'*30)
print('Checking for missing data in test:')
print('Missing values found:',test.isna().sum().sum())

In [ ]:
# parsing dates and checking for consistent trip_duration values
# train
train['pickup_datetime'] = pd.to_datetime(train['pickup_datetime'])
train['pickup_date'] = pd.to_datetime(train['pickup_datetime'].dt.date)
train['weekday'] = train['pickup_datetime'].dt.weekday
train['pickup_hour'] = train['pickup_datetime'].dt.hour
train['pickup_month'] = train['pickup_datetime'].dt.month
train['day_of_year'] = train['pickup_datetime'].dt.dayofyear
train['date'] = pd.to_datetime(train['pickup_datetime'].dt.date)
train['dropoff_datetime'] = pd.to_datetime(train['dropoff_datetime'])
train['dropoff_date'] = pd.to_datetime(train['dropoff_datetime'].dt.date)

#total duration in seconds
train['calculated_duration'] = (train['dropoff_datetime'] - train['pickup_datetime']).dt.total_seconds() 
inconsistent_trips_indices = train[(np.abs(train['calculated_duration'] - train['trip_duration'])) > 1].index

# test
test['pickup_datetime'] = pd.to_datetime(test['pickup_datetime'])
test['pickup_date'] = pd.to_datetime(test['pickup_datetime'].dt.date)
test['weekday'] = test['pickup_datetime'].dt.weekday
test['pickup_hour'] = test['pickup_datetime'].dt.hour
test['pickup_month'] = test['pickup_datetime'].dt.month
test['day_of_year'] = test['pickup_datetime'].dt.dayofyear
test['date'] = pd.to_datetime(test['pickup_datetime'].dt.date)


print(f'Inconsistent trip durations found at indices: {inconsistent_trips_indices}') if (len(inconsistent_trips_indices) > 0) else print('All Trip durations are OK!!')
train.drop(['calculated_duration'],axis=1,inplace=True)

In [ ]:
# check train and test overlap - to make sure that we are really training on features that are relevant to our test data set 
train['pickup_date'] = pd.to_datetime(pd.to_datetime(train['pickup_datetime']).dt.date)
test['pickup_date'] = pd.to_datetime(pd.to_datetime(test['pickup_datetime']).dt.date)

fig,ax1 = plt.subplots(1,1,figsize=(15,7))
plt.plot(train.groupby('pickup_date').count()[['vendor_id']], 'o-', label='train')
plt.plot(test.groupby('pickup_date').count()[['vendor_id']], 'o-', label='test')
ax1.set_title('Train and Test Trips over Time.')
ax1.legend(loc=0)
ax1.set_ylabel('Trips')

In [ ]:
city_long_border = (-74.03, -73.75)
city_lat_border = (40.63, 40.85)
fig, ax = plt.subplots(ncols=2, sharex=True, sharey=True,figsize=(15,7))
ax[0].scatter(train['pickup_longitude'].values[:50000], train['pickup_latitude'].values[:50000],
              color='purple', s=1, label='train', alpha=0.1)

ax[1].scatter(test['pickup_longitude'].values[:50000], test['pickup_latitude'].values[:50000],
              color='red', s=1, label='test', alpha=0.1)


fig.suptitle('Train and test area complete overlap.')
ax[0].legend(loc=0)
ax[0].set_ylabel('latitude')
ax[0].set_xlabel('longitude')
ax[1].set_xlabel('longitude')
ax[1].legend(loc=0)
plt.ylim(city_lat_border)
plt.xlim(city_long_border)
plt.show()

We find that our train and test data sets do indeed cover the same time range and geographical area.

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(15,4))

hist1 = sns.histplot(data = train, x=np.log(1+train['trip_duration']),ax=ax,kde=True)
hist1.set(title='Distribution of trip_duration')
fig.show()

In [ ]:
# converting trip duration into log-space
train.loc[:,'trip_duration'] = np.log(1+train.loc[:,'trip_duration'])
print('Converted trip_duration into log space:')

In [2]:
fig = plt.figure(figsize=(15,10))
for idx,col in enumerate(['vendor_id','passenger_count','store_and_fwd_flag','pickup_hour','day_of_year','pickup_month']):
    fig.add_subplot(3,2,idx+1)
    countplot = sns.countplot(data=train,x=col)
    countplot.set(title=f'{col} distribuution')
    
fig.suptitle('Categorical and discreate variables distributions')
fig.tight_layout()
fig.show()

NameError: name 'plt' is not defined

1. Vendor 2 made more trips than vendor 1
2. Most trips were made with 1 passenger
3. There were mostly N=not a store and forward trip
4. After 12pm there was a dip in the number of rides till 5am . After that there was gradual increase till 4pm evening then again a gradual increase.
5. In day_of_year feature , we can see a sudden dip in trips around January. This can be due to weather condition.For this we'll include another dataset of weather.

In [3]:
# restrincting the range of lat and long because very few cases are there when the trip is outside NYC and most trips were in Manhatten
fig,ax = plt.subplots(2,2,figsize=(15,8))

kde1 = sns.kdeplot(data=train,x='pickup_longitude',fill=True,color='green',ax=ax[0,0])
kde2 = sns.kdeplot(data=train,x='dropoff_longitude',fill=True,color='green',ax=ax[0,1])
ax[0,0].set_xlim([-76,-70])
ax[0,1].set_xlim([-76,-70])

kde3 = sns.kdeplot(data=train,x='pickup_latitude',fill=True,color='red',ax=ax[1,0])
kde4 = sns.kdeplot(data=train,x='dropoff_latitude',fill=True,color='red',ax=ax[1,1])
ax[1,0].set_xlim([40.6,41])
ax[1,1].set_xlim([40.6,41])

fig.suptitle('Latitude and Longitude distributions')
fig.tight_layout()
fig.show()

NameError: name 'plt' is not defined

In [4]:
# train
train = train[(train['pickup_longitude'] > -74.1) & (train['pickup_longitude'] < -73.7)]
train = train[(train['pickup_latitude'] > 40.6) & (train['pickup_latitude']< 40.8)]
train = train[(train['dropoff_longitude'] > -74.1) & (train['dropoff_longitude'] < -73.7)]
train = train[(train['dropoff_latitude'] > 40.6) & (train['dropoff_latitude']< 40.8)]

# test
test = test[(test['pickup_longitude'] > -74.1) & (test['pickup_longitude'] < -73.7)]
test = test[(test['pickup_latitude'] > 40.6) & (test['pickup_latitude']< 40.8)]
test = test[(test['dropoff_longitude'] > -74.1) & (test['dropoff_longitude'] < -73.7)]
test = test[(test['dropoff_latitude'] > 40.6) & (test['dropoff_latitude']< 40.8)]

f, (ax1, ax2) = plt.subplots(1, 2,sharey=True,figsize=(15,6))
train.plot(kind='scatter', x='pickup_longitude', y='pickup_latitude',
                color='yellow', 
                s=.02, alpha=.6, subplots=True, ax=ax1)
ax1.set_title("Pickups")
ax1.set_facecolor('black')
ax1.grid(False)

train.plot(kind='scatter', x='dropoff_longitude', y='dropoff_latitude',
                color='yellow',
                s=.02, alpha=.6, subplots=True, ax=ax2)
ax2.set_title("Dropoffs")
ax2.set_facecolor('black')
ax2.grid(False)

NameError: name 'train' is not defined

In [5]:
# We use PCA to transform longitude and latitude coordinates. The rotation could help for decision tree splits.
coords = np.vstack((train[['pickup_latitude', 'pickup_longitude']].values,
                    train[['dropoff_latitude', 'dropoff_longitude']].values,
                    test[['pickup_latitude', 'pickup_longitude']].values,
                    test[['dropoff_latitude', 'dropoff_longitude']].values))

pca = PCA().fit(coords)
train['pickup_pca0'] = pca.transform(train[['pickup_latitude', 'pickup_longitude']])[:, 0]
train['pickup_pca1'] = pca.transform(train[['pickup_latitude', 'pickup_longitude']])[:, 1]
train['dropoff_pca0'] = pca.transform(train[['dropoff_latitude', 'dropoff_longitude']])[:, 0]
train['dropoff_pca1'] = pca.transform(train[['dropoff_latitude', 'dropoff_longitude']])[:, 1]
test['pickup_pca0'] = pca.transform(test[['pickup_latitude', 'pickup_longitude']])[:, 0]
test['pickup_pca1'] = pca.transform(test[['pickup_latitude', 'pickup_longitude']])[:, 1]
test['dropoff_pca0'] = pca.transform(test[['dropoff_latitude', 'dropoff_longitude']])[:, 0]
test['dropoff_pca1'] = pca.transform(test[['dropoff_latitude', 'dropoff_longitude']])[:, 1]

NameError: name 'train' is not defined

### Feature relations - How features are related to each other and with target feature i.e. trip_duration

#### vendor_id and pickup_date/dropoff_date VS trip_duration

In [6]:
#pickup and dropoff trips made by each vendors on each date
fig,(ax1,ax2) = plt.subplots(2,1,figsize=(15,6))

sns.lineplot(data=train,x='pickup_date',y='trip_duration',hue='vendor_id',ax=ax1)
sns.lineplot(data=train,x='dropoff_date',y='trip_duration',hue='vendor_id',ax=ax2)
fig.tight_layout()
fig.show()

NameError: name 'plt' is not defined

#### vendor_id and weekday/pickup_hour VS trip_duration

In [7]:
fig,ax = plt.subplots(2,1,figsize=(15,7))

sns.lineplot(data=train, x='weekday',y='trip_duration',hue='vendor_id',ax=ax[0])
sns.lineplot(data=train, x='pickup_hour',y='trip_duration',hue='vendor_id',ax=ax[1])
plt.title('Mean trip durations by each vendor')
fig.tight_layout()
fig.show()

NameError: name 'plt' is not defined

1. Vendor 2 has more average trips than vendor 1 on all weekdays. The trend is same for months. Vendor id seems to be a important feature to include in the model.
2. As seen earlier, there is a sudden dip at early morning around 5am and a gradual decrease around 8pm.
3. On Sunday, the average trip duration was around 16min. for vendor 2 and 14min. for vendor 1
4. We decide to include weekday and pickup_hour in our model.

#### passenger_count and vendor_id VS trip_duration

In [8]:
fig ,ax = plt.subplots(1,2,figsize=(15,7))

box1 = sns.boxplot(data=train[train['vendor_id']==1].reset_index(), x='passenger_count', y='trip_duration',ax=ax[0])
box2 = sns.boxplot(data=train[train['vendor_id']==2].reset_index(), x='passenger_count', y='trip_duration',ax=ax[1])
box1.set(title='Passenger count of vendor 1 VS trip duration')
box2.set(title='Passenger count of vendor 2 VS trip duration')
fig.tight_layout()
fig.show()

NameError: name 'plt' is not defined

1. With passenger = 0, the trip duration was at max minimum 2.5 min. and maximum 20 min.These are outliers and should be removed.
2. Some trips has very short duration ~ 0.5 min. with both vendors.
3. When passenger count 2 and 5, vendor 2 has some trips around ~ 340hrs.
4. Vendor 1 only carried atmost 6 passengers at a time. Very less trips are there when passenger count is 7,8,9 with vendor 2.

In [9]:
print('Rows before train:',train.shape[0])
print('Rows before test:',test.shape[0])
train = train[~(train['passenger_count'] == 0)]
test = test[~(test['passenger_count'] == 0)]
print('Rows after train:',train.shape[0])
print('Rows after train:',test.shape[0])

NameError: name 'train' is not defined

#### vendor_id VS trip_duration

In [10]:
print('Trip duration stats for vendor 1: ')
vendor1_trip = train[train['vendor_id']==1]['trip_duration']
print(np.exp(vendor1_trip).describe()[['mean','50%']])
print()
print('Trip duration stats for vendor 2: ')
vendor2_trip = train[train['vendor_id']==2]['trip_duration']
print(np.exp(vendor2_trip).describe()[['mean','50%']])

fig = plt.figure(figsize=(15,7))
sns.kdeplot(data=train, x='trip_duration',hue='vendor_id')
plt.title('Trip duration distribution of both the vendors')
fig.show()

Trip duration stats for vendor 1: 


NameError: name 'train' is not defined

1. Median trip duration by both vendors are very similar whereas there is a difference in mean trip duration because of some larger duration trips done by vendor 2

#### store_and_fwd_flag and passenger_count VS trip_duration

In [11]:
fig ,ax = plt.subplots(1,2,figsize=(15,7))

box1 = sns.boxplot(data=train[train['store_and_fwd_flag']=='N'].reset_index(), x='passenger_count', y='trip_duration',ax=ax[0])
box2 = sns.boxplot(data=train[train['store_and_fwd_flag']=='Y'].reset_index(), x='passenger_count', y='trip_duration',ax=ax[1])
box1.set(title='Passenger count and store_and_fwd_flag=N VS trip duration')
box2.set(title='Passenger count and store_and_fwd_flag=Y VS trip duration')
fig.tight_layout()
fig.show()

NameError: name 'plt' is not defined

Some trips which are stored are longer than the ones which are stored.

In [12]:
kmeans_pickup = MiniBatchKMeans(n_clusters=50, random_state=2, batch_size=10000 ,n_init = 10).fit(train[['pickup_longitude','pickup_latitude']])
kmeans_drop = MiniBatchKMeans(n_clusters=50, random_state=2, batch_size=10000 ,n_init = 10).fit(train[['dropoff_longitude','dropoff_latitude']])

# train
train.loc[:,'pickup_cluster'] =  kmeans_pickup.predict(train.loc[:,['pickup_longitude','pickup_latitude']])
train.loc[:,'dropoff_cluster'] =  kmeans_drop.predict(train.loc[:,['dropoff_longitude','dropoff_latitude']])

# test
test.loc[:,'pickup_cluster'] =  kmeans_pickup.predict(test.loc[:,['pickup_longitude','pickup_latitude']])
test.loc[:,'dropoff_cluster'] =  kmeans_drop.predict(test.loc[:,['dropoff_longitude','dropoff_latitude']])


city_long_border = (-74.03, -73.75)
city_lat_border = (40.63, 40.85)

fig, ax = plt.subplots(ncols=2, nrows=1,figsize=(15,10))
ax[0].scatter(train.pickup_longitude.values, 
           train.pickup_latitude.values, s=10, lw=0,
           c=train.pickup_cluster.values, 
           cmap='tab20', 
           alpha=0.2)
ax[1].scatter(train.dropoff_longitude.values, 
           train.dropoff_latitude.values, s=10, lw=0,
           c=train.dropoff_cluster.values, 
           cmap='tab20', 
           alpha=0.2)

ax[0].set_xlim(city_long_border)
ax[0].set_ylim(city_lat_border)
ax[0].set_xlabel('Longitude')
ax[0].set_ylabel('Latitude')

ax[1].set_xlim(city_long_border)
ax[1].set_ylim(city_lat_border)
ax[1].set_xlabel('Longitude')
ax[1].set_ylabel('Latitude')

ax[0].set_title('Pickup Clusters of New York')
ax[1].set_title('Dropoff Clusters of New York')
plt.show()

NameError: name 'train' is not defined

In [13]:
centroid_pickups = pd.DataFrame(kmeans_pickup.cluster_centers_, columns = ['centroid_pick_long', 'centroid_pick_lat'])
centroid_dropoff = pd.DataFrame(kmeans_drop.cluster_centers_, columns = ['centroid_drop_long', 'centroid_drop_lat'])

# assign names to each cluster like 1,2,3,etc.
centroid_pickups['pickup_cluster'] = centroid_pickups.index
centroid_dropoff['dropoff_cluster'] = centroid_dropoff.index

# train
train = pd.merge(train, centroid_pickups, how='left', on=['pickup_cluster'])
train = pd.merge(train, centroid_dropoff, how='left', on=['dropoff_cluster'])

# test
test = pd.merge(test, centroid_pickups, how='left', on=['pickup_cluster'])
test = pd.merge(test, centroid_dropoff, how='left', on=['dropoff_cluster'])

NameError: name 'kmeans_pickup' is not defined

In [14]:
def cluster_map(df):
    
    m = folium.Map(location=[40.767937, -73.982155], zoom_start=12)
    for i in df.index:

        pick_long = centroid_pickups.iloc[i]['centroid_pick_long']
        pick_lat = centroid_pickups.iloc[i]['centroid_pick_lat']
        cluster_no = centroid_pickups.iloc[i]['pickup_cluster']
        
        pop = 'cluster no = '+str(cluster_no)

        folium.CircleMarker(location=[pick_lat,pick_long], radius=20,color='red',fill_color='#3186cc').add_to(m)
        folium.Marker(location=[pick_lat,pick_long],tooltip=pop).add_to(m)
    return m

m = cluster_map(centroid_pickups)
m

NameError: name 'centroid_pickups' is not defined

In [15]:
def haversine_(lat1, lng1, lat2, lng2):
    """function to calculate haversine distance between two co-ordinates"""
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    AVG_EARTH_RADIUS = 6371  # in km
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return(h)


def bearing_array(lat1, lng1, lat2, lng2):
    """ Calculates the angle or direction of 2 points with consideration of the roundness of earth."""
    AVG_EARTH_RADIUS = 6371  # in km
    lng_delta_rad = np.radians(lng2 - lng1)
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    return np.degrees(np.arctan2(y, x))

# train
train.loc[:, 'distance_haversine'] = haversine_(train.loc[:,'pickup_latitude'].values, train.loc[:,'pickup_longitude'].values,
                                                train.loc[:,'dropoff_latitude'].values, train.loc[:,'dropoff_longitude'].values)

train.loc[:, 'direction'] = bearing_array(train.loc[:,'pickup_latitude'].values, train.loc[:,'pickup_longitude'].values,
                                          train.loc[:,'dropoff_latitude'].values, train.loc[:,'dropoff_longitude'].values)

train.loc[:, 'center_latitude'] = (train.loc[:,'pickup_latitude'].values + train.loc[:,'dropoff_latitude'].values) / 2
train.loc[:, 'center_longitude'] = (train.loc[:,'pickup_longitude'].values + train.loc[:,'dropoff_longitude'].values) / 2

train.loc[:,'avg_speed_hvsn'] = 1000 * train.loc[:,'distance_haversine'] / np.exp(train.loc[:,'trip_duration'])

# test
test.loc[:, 'distance_haversine'] = haversine_(test.loc[:,'pickup_latitude'].values, test.loc[:,'pickup_longitude'].values,
                                                test.loc[:,'dropoff_latitude'].values, test.loc[:,'dropoff_longitude'].values)

test.loc[:, 'direction'] = bearing_array(test.loc[:,'pickup_latitude'].values, test.loc[:,'pickup_longitude'].values,
                                          test.loc[:,'dropoff_latitude'].values, test.loc[:,'dropoff_longitude'].values)

test.loc[:, 'center_latitude'] = (test.loc[:,'pickup_latitude'].values + test.loc[:,'dropoff_latitude'].values) / 2
test.loc[:, 'center_longitude'] = (test.loc[:,'pickup_longitude'].values + test.loc[:,'dropoff_longitude'].values) / 2

NameError: name 'train' is not defined

#### havsine distance VS trip_duration

In [16]:
plt.figure(figsize=(15,5))
sns.scatterplot(data = train, x= np.log(1+train['distance_haversine']),y='trip_duration')
plt.title('Change in trip duration with distance(km)')
plt.show()

NameError: name 'plt' is not defined

1. We can see that with an increase distance(km) , trip duration is also increasing
2. There are some trips with very short distance < 1 km but very high trip duration > 50min. We should remove such extreme and suspecious points

In [17]:
# train
train.loc[:,'distance_haversine'] = np.log(1+train.loc[:,'distance_haversine'])
train.loc[:,'avg_speed_hvsn'] = np.log(1+train.loc[:,'avg_speed_hvsn'])

# test
test.loc[:,'distance_haversine'] = np.log(1+test.loc[:,'distance_haversine'])

NameError: name 'train' is not defined

In [18]:
print('Total rows before train:', train.shape[0])
print('Total rows before test:', test.shape[0])

train = train[(train['trip_duration'] > 3) & (train['trip_duration'] < 10)]
train = train[~(train['distance_haversine'] > 5)]
train = train[~((train['distance_haversine'] > 6) & (train['trip_duration'] < 6))]

test = test[~(test['distance_haversine'] > 5)]

print('Total rows after train:', train.shape[0])
print('Total rows after test:', test.shape[0])

NameError: name 'train' is not defined

In [19]:
plt.figure(figsize=(15,5))
sns.scatterplot(data = train, x='distance_haversine',y='trip_duration')
plt.title('Change in log-trip duration with log-distance(km)')
plt.show()

NameError: name 'plt' is not defined

In [20]:
fig = plt.figure(figsize=(15,5))
sns.histplot(data=train, x='avg_speed_hvsn')
plt.title('Average log-speed distribution')

NameError: name 'plt' is not defined

In [21]:
fig,ax = plt.subplots(1,2,figsize=(15,5))
avg_speed_every_hour = train.groupby(['pickup_hour','vendor_id'])['avg_speed_hvsn'].median().reset_index()
avg_speed_every_day = train.groupby(['weekday','vendor_id'])['avg_speed_hvsn'].median().reset_index()
sns.lineplot(data = avg_speed_every_hour ,x = 'pickup_hour' , y='avg_speed_hvsn',hue='vendor_id',ax=ax[0])
sns.lineplot(data = avg_speed_every_day ,x = 'weekday' , y='avg_speed_hvsn',hue='vendor_id',ax=ax[1])
ax[0].set_title('Log-Average speed on each hour')
ax[1].set_title('Log-Average speed on each day')

NameError: name 'plt' is not defined

1. Taxi travelling faster on weekends than any other day
2. During the work hours the speed is slow
3. We should make a new feature 'work' which denotes 8-6 pm on Mon to Friday

In [22]:
weather = pd.read_csv('/kaggle/input/weather-data-in-new-york-city-2016/weather_data_nyc_centralpark_2016(1).csv')
weather['date'] = pd.to_datetime(weather['date'],dayfirst=False,format='mixed')
weather.head()

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/weather-data-in-new-york-city-2016/weather_data_nyc_centralpark_2016(1).csv'

In [23]:
weather['precipitation'] = weather['precipitation'].str.replace('T','0.01').astype(float)
weather['snow fall'] = weather['snow fall'].str.replace('T','0.01').astype(float)
weather['snow depth'] = weather['snow depth'].str.replace('T','0.01').astype(float)

NameError: name 'weather' is not defined

In [24]:
train = pd.merge(train,weather,on='date')
test = pd.merge(test,weather,on='date')
print('Train shape:',train.shape)
print('Test shape:',test.shape)

NameError: name 'train' is not defined

In [25]:
fig , ax = plt.subplots(4,1,figsize=(15,7),sharex=True)
# avg_speed_dates = total2.groupby(['date'])['avg_speed_hvsn'].median().reset_index()
trips_per_day = pd.DataFrame(train.groupby(['date'])['date'].count()).rename(columns={'date':'trip_count'}).reset_index()
sns.lineplot(data=train, x='date',y='precipitation',ax=ax[0],color='orange')
sns.lineplot(data=train, x='date',y='snow fall',ax=ax[1],color='red')
sns.lineplot(data=train, x='date',y='snow depth',ax=ax[2],color='purple')
sns.lineplot(data=trips_per_day, x='date',y='trip_count',ax=ax[3],color='green')

fig.tight_layout()

NameError: name 'plt' is not defined

We can see a spike between the month of January and Feburary of 2016 for high snowfall and precipitation and strong dip in passenger count at the same time which is indeed true.More can be read from [here](https://www.weather.gov/okx/Blizzard_Jan2016) 

In [26]:
# convert to numerical
train.store_and_fwd_flag = (train.store_and_fwd_flag=='Y').astype(int)
test.store_and_fwd_flag = (test.store_and_fwd_flag=='Y').astype(int)

NameError: name 'train' is not defined

In [27]:
sns.set(style="white")

# Generate a large random dataset
temp3 = train.copy()
temp3.drop(['id'],axis=1,inplace=True)

# Compute the correlation matrix
corr = temp3.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 13))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

NameError: name 'sns' is not defined

In [28]:
y = train['trip_duration']
train_id = train['id']
test_id = test['id']
train = train.drop(['id', 'pickup_datetime','dropoff_datetime','trip_duration', 'pickup_date','dropoff_date','date','avg_speed_hvsn'],axis=1)
test = test.drop(['id', 'pickup_datetime', 'pickup_date','date',],axis=1)
print(train.shape,test.shape)

NameError: name 'train' is not defined

In [29]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
y = y.reset_index(drop=True)
# train-test split
X_train, X_val, y_train, y_val = train_test_split(train,y,test_size=0.2,random_state=2)
print(X_train.shape,y_train.shape)
print(X_val.shape,y_val.shape)

NameError: name 'train' is not defined

In [30]:
regressors = {
    "LGBM1" : LGBMRegressor(random_state=2),
    "LGBM2" : LGBMRegressor(random_state=2)
}

## Grids for randomized search
lgbm_grid1 = {
        'n_estimators': np.arange(100, 1000, 100),
        'learning_rate': np.arange(0.01, 0.3, 0.01),
        'max_depth': np.arange(3, 11),
        'subsample': np.arange(0.6, 1.0, 0.05),
        'colsample_bytree': np.arange(0.6, 1.0, 0.05),
        'reg_alpha': np.arange(0.0, 1.0, 0.1),
        'reg_lambda': np.arange(0.0, 1.0, 0.1),
        }

lgbm_grid2 = {
        'n_estimators': np.arange(100, 1000, 200),
        'learning_rate': np.arange(0.01, 0.3, 0.05),
        'max_depth': np.arange(3, 11),
        'subsample': np.arange(0.6, 1.0, 0.1),
        'colsample_bytree': np.arange(0.6, 1.0, 0.1),
        'reg_alpha': np.arange(0.0, 1.0, 0.2),
        'reg_lambda': np.arange(0.0, 1.0, 0.2),
}


# # Dictionary of all grids
grid = {
    "LGBM1" : lgbm_grid1,
    "LGBM2" : lgbm_grid2,
}


reg_best_params=regressors.copy() #store best parameters
valid_scores=pd.DataFrame({'Regressors':regressors.keys(),
                           'Validation accuracy': np.zeros(len(regressors)),
                           'Training time': np.zeros(len(regressors))})

for i , (key, regressor) in enumerate(regressors.items()):
    start = time.time()
    reg = RandomizedSearchCV(estimator=regressor, param_distributions=grid[key], n_jobs=-1, cv=2,verbose=1,n_iter=3)

    # Train and save the scores
    reg.fit(X_train, y_train)
    valid_scores.iloc[i,1]=reg.score(X_val, y_val)

    # Save best parameters of trained model
    reg_best_params[key]=reg.best_params_
    
    # Print iteration and training time
    stop = time.time()
    valid_scores.iloc[i,2]=np.round((stop - start)/60, 2)
    
    print('Model:', key)
    print('Training time (mins):', valid_scores.iloc[i,2])
    print('')

NameError: name 'LGBMRegressor' is not defined

In [31]:
valid_scores

NameError: name 'valid_scores' is not defined

In [32]:
#  regressors
regressors = {
    "LGBM1" : LGBMRegressor(**reg_best_params["LGBM1"], random_state=2),
    "LGBM2" : LGBMRegressor(**reg_best_params["LGBM2"], random_state=2)
}

NameError: name 'LGBMRegressor' is not defined

In [33]:
# Stacking ensemble technique - a method for combining estimators to reduce their biases.More precisely, the predictions of each individual estimator are stacked together and used as input to a final estimator to compute the prediction. This final estimator is trained through cross-validation.
estimators = [('lgbm1', regressors['LGBM1'])]
final_estimator = regressors['LGBM2']
model = StackingRegressor(
    estimators=estimators,
    final_estimator=final_estimator,verbose=1,n_jobs=-1,cv=5)

# train model
model.fit(X_train,y_train)

NameError: name 'regressors' is not defined

In [34]:
print('Accuracy of model on Validation samples: ',model.score(X_val,y_val))
final_pred = model.predict(test)
pred = np.exp(final_pred)

NameError: name 'model' is not defined

In [35]:
# submission
submission = pd.DataFrame({
    'id': test_id.values,
    'trip_duration': pred})

submission.to_csv('submission.csv')

NameError: name 'test_id' is not defined